In [21]:
import pandas as pd

# Merging on a specific column

(**pd.concat()** nos permite concatener DataFrames de forma horizontal y vertical, por otro lado **df.append()** nos permite concatenar DataFrames de forma horizontal. Existen situaciones en las que deseamos concatenar conjuntos de datos mediante valores en diversas columnas. Para realizar esto contamos con **pd.merge()**. Esta función por defecto realiza un join por valores similares entre columnas.

In [5]:
#Cargamos los datos
revenue = pd.read_csv('revenue.csv')
print(revenue)

   branch_id         city  revenue
0         10       Austin      100
1         20       Denver       83
2         30  Springfield        4
3         47    Mendocino      200


In [6]:
managers = pd.read_csv('manager.csv')
print(managers)

   branch_id         city  manager
0         10       Austin  Charles
1         20       Denver     Joel
2         47    Mendocino    Brett
3         31  Springfield    Sally


In [7]:
#Hacemos el merge
print(pd.merge(revenue, managers))

   branch_id       city  revenue  manager
0         10     Austin      100  Charles
1         20     Denver       83     Joel
2         47  Mendocino      200    Brett


Si queremos especificar la unión a partir de una determinada columna, podemos hacer uso del parámetro **on**, mediante el cual indicamos el nombre de la columna a partir de la cual queremos hacer la unión. La columna especificada debe estar en ambos conjuntos de datos.

In [8]:
#Hacemos el merge por la columna city
merge_by_city  = pd.merge(revenue, managers, on = 'city')
print(merge_by_city)

   branch_id_x         city  revenue  branch_id_y  manager
0           10       Austin      100           10  Charles
1           20       Denver       83           20     Joel
2           30  Springfield        4           31    Sally
3           47    Mendocino      200           47    Brett


In [12]:
#Hacemos el merfe por la columna branch_id 
merge_by_id = pd.merge(revenue, managers, on = ['branch_id'])
print(merge_by_id)

   branch_id     city_x  revenue     city_y  manager
0         10     Austin      100     Austin  Charles
1         20     Denver       83     Denver     Joel
2         47  Mendocino      200  Mendocino    Brett


Podemos especificar mediante el argumento **suffixes** el nombre de las columnas creadas.

In [14]:
merge_by_id = pd.merge(revenue, managers, on = ['branch_id'], suffixes=['_revenue', '_managers'])
print(merge_by_id)

   branch_id city_revenue  revenue city_managers  manager
0         10       Austin      100        Austin  Charles
1         20       Denver       83        Denver     Joel
2         47    Mendocino      200     Mendocino    Brett


# Merging on columns with non-matching labels

Es posible que se de la situación en la que las columnas a partir de las cuales queremos realizar la unión no tengan el mismo nombre. En este caso **pd.merge()** dispone de los argumentes **left_on** y **right_on** donde le indiacamos las columnas por las cuales queremos realizar la unión.

In [16]:
#Cargamos los datos
revenue = pd.read_csv('revenue.csv')
print(revenue)

   branch_id         city  revenue
0         10       Austin      100
1         20       Denver       83
2         30  Springfield        4
3         47    Mendocino      200


In [19]:
managers2 = pd.read_csv('managers2.csv')
print(managers2)

        branch  branch_id   manager state
0       Austin         10  Charlers    TX
1       Denver         20      Joel    CO
2    Mendocino         47     Brett    CA
3  Springfield         31     Sally    MO


In [20]:
#Hacemos la unión por las columnas city y branch
print(pd.merge(revenue, managers2, left_on = 'city', right_on = 'branch'))

   branch_id_x         city  revenue       branch  branch_id_y   manager state
0           10       Austin      100       Austin           10  Charlers    TX
1           20       Denver       83       Denver           20      Joel    CO
2           30  Springfield        4  Springfield           31     Sally    MO
3           47    Mendocino      200    Mendocino           47     Brett    CA


# Merging on multiple columns

Una forma de evitar la situación de que se repitan columnas es hacer un merge por diversas columnas.

In [21]:
#Nos creamos la columnas state en revenue y mangers
revenue['state'] = ['TX', 'CO', 'IL', 'CA']
print(revenue) 

   branch_id         city  revenue state
0         10       Austin      100    TX
1         20       Denver       83    CO
2         30  Springfield        4    IL
3         47    Mendocino      200    CA


In [22]:
managers['state'] = ['TX', 'CO', 'CA', 'MO']
print(managers)

   branch_id         city  manager state
0         10       Austin  Charles    TX
1         20       Denver     Joel    CO
2         47    Mendocino    Brett    CA
3         31  Springfield    Sally    MO


In [24]:
#Hacemos el merge
print(pd.merge(revenue, managers, on = ['branch_id', 'city', 'state']))

   branch_id       city  revenue state  manager
0         10     Austin      100    TX  Charles
1         20     Denver       83    CO     Joel
2         47  Mendocino      200    CA    Brett


# Left & right merging on multiple columns

In [26]:
#Cargamos datos
sales = pd.read_csv('sales.csv')
print(sales)

          city state  units
0    Mendocino    CA      1
1       Denver    CO      4
2       Austin    TX      2
3  Springfield    MO      5
4  Springfield    IL      1


El conjunto de datos sales dispone de la información de unidades vendidas en cada una de las tiendas. Si deseamos conocer aquellas ventas de las que no conocemos la ganancia podemos realizar un merge entre revenue y sales.

In [31]:
sales_revenue = pd.merge(sales, revenue, on = ['city', 'state'], how = 'left')
print(sales_revenue)

          city state  units  branch_id  revenue
0    Mendocino    CA      1       47.0    200.0
1       Denver    CO      4       20.0     83.0
2       Austin    TX      2       10.0    100.0
3  Springfield    MO      5        NaN      NaN
4  Springfield    IL      1       30.0      4.0


Si deseamos conocer las ventas asociados a cada manager poodemos realizar la siguiente operación.

In [32]:
sales_managers = pd.merge(sales, managers2, left_on = ['city', 'state'], right_on = ['branch', 'state'], how = 'left')
print(sales_managers)

          city state  units       branch  branch_id   manager
0    Mendocino    CA      1    Mendocino       47.0     Brett
1       Denver    CO      4       Denver       20.0      Joel
2       Austin    TX      2       Austin       10.0  Charlers
3  Springfield    MO      5  Springfield       31.0     Sally
4  Springfield    IL      1          NaN        NaN       NaN


# Merging DataFrames with outer join

A continuación vamos a proceder a juntar toda la información en único dataframe.

In [35]:
print(pd.merge(sales_revenue, sales_managers, on = ['city', 'state'], how = 'outer'))

          city state  units_x  branch_id_x  revenue  units_y       branch  \
0    Mendocino    CA        1         47.0    200.0        1    Mendocino   
1       Denver    CO        4         20.0     83.0        4       Denver   
2       Austin    TX        2         10.0    100.0        2       Austin   
3  Springfield    MO        5          NaN      NaN        5  Springfield   
4  Springfield    IL        1         30.0      4.0        1          NaN   

   branch_id_y   manager  
0         47.0     Brett  
1         20.0      Joel  
2         10.0  Charlers  
3         31.0     Sally  
4          NaN       NaN  


# Using merge_ordered()

**pd.merge_ordered()** nos permite ordenar el DataFrame unido por el campo especificado. Por defecto el tipo de join realizado es **outer**. También dispone del argumento fill_method, para en caso de que obtengamos un valor perdido decirle la forma en la cual deseamos rellenar dicho valor.

In [5]:
#Cargamos los datos
austin = pd.read_csv('austin_weather.csv')
print(austin)

         date ratings
0  2016-01-01  Cloudy
1  2016-02-08  Cloudy
2  2016-01-17   Sunny


In [6]:
houston = pd.read_csv('houston_weather.csv')
print(houston)

         date ratings
0  2016-01-04   Rainy
1  2016-01-01  Cloudy
2  2016-03-01   Sunny


In [7]:
#Hacemos uso de pd_merge_ordered
print(pd.merge_ordered(austin, houston))

         date ratings
0  2016-01-01  Cloudy
1  2016-01-04   Rainy
2  2016-01-17   Sunny
3  2016-02-08  Cloudy
4  2016-03-01   Sunny


In [9]:
#Hacemos uso de pd.merge_ordered con sus argumentos
print(pd.merge_ordered(austin, houston, on = ['date'], suffixes = ['_austin', '_houston']))

         date ratings_austin ratings_houston
0  2016-01-01         Cloudy          Cloudy
1  2016-01-04            NaN           Rainy
2  2016-01-17          Sunny             NaN
3  2016-02-08         Cloudy             NaN
4  2016-03-01            NaN           Sunny


In [10]:
#Hacemos uso de pd.merge_ordered con sus argumentos
print(pd.merge_ordered(austin, houston, on = ['date'], suffixes = ['_austin', '_houston'], fill_method = 'ffill'))

         date ratings_austin ratings_houston
0  2016-01-01         Cloudy          Cloudy
1  2016-01-04         Cloudy           Rainy
2  2016-01-17          Sunny           Rainy
3  2016-02-08         Cloudy           Rainy
4  2016-03-01         Cloudy           Sunny
